## 작업형 제 2 유형

### ✏️ 여행 보험 패키지 상품을 구매할 확률값을 예측하시오.
- 제공된 데이터 목록 : train.csv, test.csv

- 예측할 컬럼 : TravelInsurance(여행 보험 패키지를 구매했는지 여부, 0 : 구매 안함, 1 : 구매)

- 학습용 데이터(train)을 이용하여 여행 보험 패키지 상품을 구매할 예측 모형을 만든 후 이를 평가용 데이터(test)에 적용하여 얻은 예측(가입 확률)값을 다음과 같은 형식의 csv 파일로 생성하시오. (제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점)

    - 제출 csv 파일명 : result.csv

    - 형태 : index, y_pred  컬럼

In [1]:
# 1. 문제 정의
## 여행 보험 패키지 데이터, 여행 보험 패키지 상품(TravelInsurance) 구매 여부 예측
## - 평가 기준은 ROC-AUC
## - label(target) : 구매여부(TravelInsurance) 0: 구매 안함, 1: 구매
## - 제출 방식은 test 데이터로 구매 확률을 csv 로 제출(파일명 : result.csv)

# 2. 라이브러리 및 데이터 불러오기
import pandas as pd
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

# 3. 탐색적 데이터 분석(EDA)
## 데이터 크기 확인
print(train.shape, test.shape, '\n')

## train 데이터 확인 : 카테고리(문자)와 연속형(숫자) 컬럼 혼합
## 'Unnamed: -' 이라는 알 수 없는 컬럼, TravelInsurance 라는 label 이 숫자로 된 것 확인
print(train.head(3), '\n')

## 데이터 자료형(타입) : int64 6개, object 4개 -> object 컬럼은 인코딩 필요
print(train.info(), '\n')

## 기초 통계 값
## 'Unnamed: 0' 컬럼은 데이터 수와 종류가 1,490 으로 같으므로 큰 의미가 없어 전처리에서 삭제해도 삭제하지 않아도 ok
## AnnualIncome 지수 표기법으로 표기되어있어 보기 어려우므로 판다스 설정 변경 -> '.4f' : 소수 넷째 자리까지 표기하라고 설정하는 것
print(train.describe(), '\n')
print(train['Unnamed: 0'].nunique(), '\n')

pd.options.display.float_format = '{:.4f}'.format
print(train.describe(), '\n')

## object 컬럼의 unique 개수 : 모두 2개씩 카테고리를 가짐
print(train.describe(include='object'), '\n')

## test 데이터셋에 있는 object 컬럼 확인 : train 데이터와 유사한 형타
print(test.describe(include='O'), '\n')

## 결측치 확인 : train, test 데이터에 결측치 없음
print(train.isnull().sum(), '\n')
print(test.isnull().sum().sum(), '\n')

## TravelInsurance 컬럼 종류에 따른 개수 확인
print(train['TravelInsurance'].value_counts())

(1490, 10) (497, 9) 

   Unnamed: 0  Age               Employment Type GraduateOrNot  AnnualIncome  \
0         888   28  Private Sector/Self Employed           Yes       1250000   
1        1308   31  Private Sector/Self Employed           Yes       1250000   
2         151   29  Private Sector/Self Employed           Yes       1200000   

   FamilyMembers  ChronicDiseases FrequentFlyer EverTravelledAbroad  \
0              6                1            No                  No   
1              7                1            No                  No   
2              7                0            No                  No   

   TravelInsurance  
0                0  
1                0  
2                1   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           1490 non-null   int64 
 1   Age                  1490 n

In [2]:
# 4. 데이터 전처리
## target 컬럼을 변수에 옮겨두고 자료형이 object 인 컬럼을 판다스에서 제공하는 함수를 활용해 원-핫 인코딩
## pd.get_dummies() : 범주형 데이터만 자동으로 원-핫 인코딩 진행
## 베이스라인에서는 레이블 인코딩 또는 원-핫 인코딩 중에서 좀 더 편한 것으로 작업하면 됨
## 원-핫 인코딩 전과 후의 컬럼의 변화를 눈으로 확인  후 차이가 있다면 아래와 같이 의심
## - train 과 test 범주형 컬럼의 카테고리 수가 다름 -> train 과 test 데이터를 합쳐서 인코딩 진행
## - label 이 숫자가 아닌 범주형 데이터로 원-핫 인코딩 처리됨 -> label 데이터를 분리하고 인코딩 진행
target = train.pop('TravelInsurance')
print(train.shape, test.shape, '\n')

train = pd.get_dummies(train)
test = pd.get_dummies(test)
print(train.shape, test.shape)

(1490, 9) (497, 9) 

(1490, 13) (497, 13)


---

#### 💡 train 과 test 범주형 컬럼의 카테고리 수가 다르다면?
- train 과 test 데이터를 합쳐서 이노딩하고 다시 분할

    - train 과 test 는 컬럼 수가 다르므로 합했다가 분할했을 때 test 의 'TravelInsurance' 컬럼에 생성됨

        - test 에서 'TravelInsurance' 제거

    - 데이터 누출(Data Leekage) 문제 야기할 수 있음

        - test 데이터에만 있는 카테고리가 있을 경우 카테고리 학습 정보가 머신러닝 학습 과정에 포함되어 성능엥 영향을 미칠 수 있기 때문

- 타깃 'TravelInsurance' 컬럼을 train 에서 미리 분리하고, 데이터 합치기

---

In [3]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

target = train.pop('TravelInsurance')
print(train.shape, test.shape, '\n')

combined = pd.concat([train, test])
combined_dummies = pd.get_dummies(combined)
n_train = len(train)
train = combined_dummies[:n_train]
test = combined_dummies[n_train:]

print(train.shape, test.shape)

print(train.columns)

(1490, 9) (497, 9) 

(1490, 13) (497, 13)
Index(['Unnamed: 0', 'Age', 'AnnualIncome', 'FamilyMembers', 'ChronicDiseases',
       'Employment Type_Government Sector',
       'Employment Type_Private Sector/Self Employed', 'GraduateOrNot_No',
       'GraduateOrNot_Yes', 'FrequentFlyer_No', 'FrequentFlyer_Yes',
       'EverTravelledAbroad_No', 'EverTravelledAbroad_Yes'],
      dtype='object')


In [4]:
# 5. 검증 데이터 분할
## train 데이터를 활용해 검증 데이터(20%) 분할
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(1192, 13) (298, 13) (1192,) (298,)


In [5]:
# 6. 머신러닝 학습 및 평가
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
!pip install xgboost
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import roc_auc_score   # roc-auc 는 1에 가까울수록 좋은 모델

## 1. 로지스틱 회귀 모델
lr = LogisticRegression(random_state=0)
lr.fit(X_train, y_train)
pred = lr.predict_proba(X_val)
print('로지스틱 회귀 :', roc_auc_score(y_val, pred[:,1]))

## 2. 의사결정 나무 모델
dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_train, y_train)
pred = dt.predict_proba(X_val)
print('의사결정 나무 :', roc_auc_score(y_val, pred[:,1]))

## 3. 랜덤포레스트 모델
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)
pred = rf.predict_proba(X_val)
print('랜덤포레스트 :', roc_auc_score(y_val, pred[:,1]))

## 4. xgboost 모델
xg = xgb.XGBClassifier(random_state=0)
xg.fit(X_train, y_train)
pred = xg.predict_proba(X_val)
print('xgboost :', roc_auc_score(y_val, pred[:,1]))

## 5. lightGBM 모델 -> LightGBMError: Do not support special JSON characters in feature name.
### 컬럼명의 특수 문자 ":" 때문에 발생. 컬럼명을 rename() 함수를 활용해 변경 후 재실행
### 만약, 'Unnamed: 0' 컬럼을 삭제했다면 해당 에러 발생하지 않았을 것
X_train.rename(columns={'Unnamed: 0' : 'Unnamed'}, inplace=True)
X_val.rename(columns={'Unnamed: 0' : 'Unnamed'}, inplace=True)

lg = lgb.LGBMClassifier(random_state=0)
lg.fit(X_train, y_train)
pred = lg.predict_proba(X_val)
print('lightGBM :', roc_auc_score(y_val, pred[:,1]))

c:\Users\babys\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


로지스틱 회귀 : 0.7743664717348929
의사결정 나무 : 0.726803118908382
랜덤포레스트 : 0.8506578947368421
xgboost : 0.8084795321637427
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 417, number of negative: 775
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001071 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 324
[LightGBM] [Info] Number of data points in the train set: 1192, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.349832 -> initscore=-0.619777
[LightGBM] [Info] Start training from score -0.619777
lightGBM : 0.8198830409356724


In [10]:
# 7. 예측 및 결과 파일 생성
## 제출 양식 형태로 컬럼명을 작성하고 데이터프레임 만들기
## index 컬럼은 test 데이터의 인덱스로 y_pred 컬럼은 1 확률값만 대입한 다음 csv 파일로 생성
## index 파라미터는 반드시 False 로 진행
pred = rf.predict_proba(test)
submit = pd.DataFrame({'index':test.index, 'y_pred':pred[:,1]})
submit.to_csv('result.csv', index=False)

pd.read_csv('result.csv').head(3)

,index,y_pred
0,0,0.2400
1,1,0.0800
2,2,0.1200


In [ ]:
# 8-1. 성능 개선
## 인코딩 진행

# 데이터 불러오기
import pandas as pd
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

target = train.pop('TravelInsurance')

# 레이블 인코딩 => 0.8610380116959064(원-핫 인코딩시 0.8506578947368421 이므로 레이블 인코딩 채택)
from sklearn.preprocessing import LabelEncoder
cols = train.select_dtypes(include='object').columns
for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])
    
# 검증 데이터 분할
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)
pred = rf.predict_proba(X_val)
print(roc_auc_score(y_val, pred[:,1]))

(1192, 9) (298, 9) (1192,) (298,)
0.8610380116959064


In [ ]:
# 8-2. 성능 개선
## 스케일링 진행

# 데이터 불러오기
import pandas as pd
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

target = train.pop('TravelInsurance')

# 스케일링 : 스케일링 전 0.8610380116959064
from sklearn.preprocessing import StandardScaler    # 0.860233918128655
from sklearn.preprocessing import MinMaxScaler      # 0.8606481481481483
from sklearn.preprocessing import RobustScaler      # 0.8612573099415205 채택
scaler = RobustScaler()
cols = train.select_dtypes(exclude='object').columns
train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.transform(test[cols])

# 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
cols = train.select_dtypes(include='object').columns
for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])
    
# 검증 데이터 분할
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)
pred = rf.predict_proba(X_val)
print(roc_auc_score(y_val, pred[:,1]))

(1192, 9) (298, 9) (1192,) (298,)
0.8612573099415205


In [16]:
# 9. 두 번째 csv 파일 생성 후 제출
pred = rf.predict_proba(test)
submit = pd.DataFrame({'index':test.index, 'y_pred':pred[:,1]})
submit.to_csv('result.csv', index=False)

print(pd.read_csv('result.csv').head(3))

   index  y_pred
0      0  0.2100
1      1  0.0900
2      2  0.1000
